# Detecting Extreme Precipitation Events

In [ ]:
from glob import glob
import xarray as xr
# Cosmetic options
from rich import print
%load_ext rich

BASE_PATH = '/scratch/midway3/nnn/'

In [ ]:
ds_cp = xr.open_mfdataset(f'{BASE_PATH}*_cp.nc')
ds_cape = xr.open_mfdataset(f'{BASE_PATH}*_cape.nc')
ds_lsp = xr.open_mfdataset(f'{BASE_PATH}*_lsp.nc')